## QA h3 calculations on Water Risk and Impact.

This notebooks contains the QA of the h3 calculations performed on the ddbb and stimates the error we may be having from those.

1. Crop production:

    1.1 Transform crop production to h3  
    1.2 Get the crop production in h3 from the ddbb  
    1.3 Compute differences  
    
    
2. Risk map:

    2.1 Transform rik map for location x to h3  
    2.2 Get the calculation of risk map in h3 from the database  
    2.3 Compute differences  
    
    
3. Impact map:

    3.1 Search for a sourcing record for purchased of cotton in india and get the purchased volume  
    3.2 Calculatethe average of water risk in india for cotton  
    3.3 Multiply that average value times the purchased volume  
    3.4 Compare that value with the associated indicator record value for water risk.  
    
    
   

In [2]:
# import libraries
from psycopg2.pool import ThreadedConnectionPool

In [3]:
## env file for gcs upload
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

list(env.keys())

['API_SERVICE_PORT',
 'API_POSTGRES_HOST',
 'API_POSTGRES_PORT',
 'API_POSTGRES_USERNAME',
 'API_POSTGRES_PASSWORD',
 'API_POSTGRES_DATABASE',
 'CLIENT_SERVICE_PORT']

In [4]:
postgres_thread_pool = ThreadedConnectionPool(
    1,
    50,
    host=env["API_POSTGRES_HOST"],
    port=env["API_POSTGRES_PORT"],
    user=env["API_POSTGRES_USERNAME"],
    password=env["API_POSTGRES_PASSWORD"],
)

## Crop production

In [10]:
material = "spam2010V2R0GlobalPCottA"
sql = f"""select prod.h3index, prod."{material}"
    from h3_grid_spam2010v2r0_global_prod prod 
    where prod.h3index in ('86c203047ffffff', '86c20304fffffff')"""
sql

'select prod.h3index, prod."spam2010V2R0GlobalPCottA"\n    from h3_grid_spam2010v2r0_global_prod prod \n    where prod.h3index in (\'86c203047ffffff\', \'86c20304fffffff\')'

In [11]:
# connect to the ddbb
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()


## NOTE: The same logic for the and indicators, materials and admin regions would be applied to the supplier.
# As all the data is null, I'm not filtering by anything in this case
cursor.execute(sql)

response = cursor.fetchall()

In [12]:
response

[('86c20304fffffff', 0.0), ('86c203047ffffff', 0.0)]

## Risk map:

In [1]:
sql = """select mprod.h3index, (windicator."wfBltotMmyr" * 0.001)/sum(mprod."spam2010V2R0GlobalPCottA") over() wr_m3_t 
    from h3_grid_spam2010v2r0_global_prod mprod
    inner join h3_grid_wf_global wIndicator on wIndicator.h3index = mprod.h3index 
    where windicator."wfBltotMmyr" is not null
    and mprod."spam2010V2R0GlobalPCottA" is not null"""

## Impact map:

In [2]:
sql = """select sr.tonnage, sr."year", sl."locationCountryInput", m."name",i2."shortName", ir.value from sourcing_records sr 
left join sourcing_location sl on sl.id =sr."sourcingLocationId" 
left join material m on m.id =sl."materialId" 
right join indicator_record ir on ir."sourcingRecordId" = sr.id 
left join "indicator" i2 on i2.id =ir."indicatorId" 
where sl."locationCountryInput"='India'
and m."name" = 'Cotton'
and sr."year"=2010
and i2."shortName" = 'Unsustainable water use'"""